# Segmenting and Clustering Neighborhoods in Toronto
_Week 3 Project_

**To Do:**
 * Increase limit
 * Create link to notebook using: https://nbviewer.jupyter.org/

**Goals:**
* Scrape neighborhood data from Wikipedia page
* Retrieve lat/lng coordinates for each neighborhood
* Explore and cluster neighborhoods

#### Import Required Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#### Scrape Wiki Page

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'html.parser')
# print(soup.prettify()) # Uncomment to view scraped HTML

In [4]:
table = soup.find("table", class_="wikitable")
table_body = table.tbody
table_rows = table_body.findAll("tr")

#### Process Table HTML

In [5]:
cols = ["PostalCode", "Borough", "Neighborhood"]
data = []

for row in table_rows:
    cells = row.findAll("td")
    if len(cells) != 0:
        postcode = cells[0].text.strip()
        borough = cells[1].text.strip()
        neighborhood = cells[2].text.strip()
        if borough != "Not assigned":
            if neighborhood == "Not assigned":
                neighborhood = borough
            data.append([postcode, borough, neighborhood])
df = pd.DataFrame(data)
df.columns = cols
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### Group by PostalCode

In [6]:
# Technique 1
df_grouped = df.groupby(['PostalCode','Borough'])['Neighborhood'].agg(lambda col: ', '.join(col))

# Technique 2
# df_grouped = df.groupby(['PostalCode','Borough']).Neighborhood.unique().apply(lambda x: ', '.join(x)).reset_index()

toronto = df_grouped.to_frame().reset_index()
toronto

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**Check Number of Rows**

In [7]:
print("The Toronto dataframe has {} rows.".format(toronto.shape[0]))

The Toronto dataframe has 103 rows.


#### Get Coordinates of Postal Codes
(Imported from csv as geocoder did not work for me)

In [9]:
## Add two empty columns for lat/lng
toronto["Latitude"], toronto["Longitude"] = np.nan, np.nan

**Attempted to use Geocoder:**

In [10]:
import geocoder

In [11]:
def get_coords(postal_code):
    coords = None

    while (coords is None):-
        g = geocoder.google("{}, Toronto, Ontario".format(postal_code))
        coords = g.latlng

    lat = coords[0]
    lng = coords[1]
    return lat, lng

In [12]:
## Apply get coords function to each postal code ** Did not work **
# toronto["Latitude"], toronto["Longitude"] = zip(*toronto["PostalCode"].apply(get_coords))

**Pulled Lat/Lngs from CSV**

In [13]:
coords = pd.read_csv("assets/Geospatial_Coordinates.csv").set_index("Postal Code")
coords.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [14]:
unique_postal_codes = toronto["PostalCode"].unique()
toronto = toronto.set_index("PostalCode")

for p in unique_postal_codes:
    toronto.loc[p, "Latitude"], toronto.loc[p, "Longitude"] = coords.loc[p, "Latitude"], coords.loc[p, "Longitude"]
    
toronto = toronto.reset_index()

In [15]:
toronto.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Get the Lat/Lng of Toronto

In [16]:
from geopy.geocoders import Nominatim
import folium

In [17]:
address = "Toronto, Ontario"

geolocator = Nominatim(user_agent="ca_user")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


#### Create Map Plotting Helper Function

In [18]:
def plot_locations(center_lat, center_lng, locations_df):
    toronto_map = folium.Map(location=[center_lat, center_lng], zoom_start=11)

    folium.vector_layers.CircleMarker(
        [center_lat, center_lng],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        tooltip="Toronto"
    ).add_to(toronto_map)

    for lat, lng, label in zip(locations_df["Latitude"], locations_df["Longitude"], locations_df["Neighborhood"]):
        folium.vector_layers.CircleMarker(
            [lat, lng],
            radius=3,
            color="blue",
            fill=True,
            fill_color="red",
            fill_opacity=0.6,
            popup=label
        ).add_to(toronto_map)
        
    return toronto_map

#### Map All Neighborhoods

In [19]:
plot_locations(latitude, longitude, toronto)

#### For Simplicity & Speed, Let's Look at a Subset of the Boroughs

In [63]:
toronto_data = toronto[toronto["Borough"].str.contains("Toronto")]
print(toronto_data.shape)
toronto_data.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [39]:
plot_locations(latitude, longitude, toronto_data)

#### Get Nearby Venues for Each Borough using Foursquare API

In [22]:
# import Foursquare developer credentials
%run "credentials.py"

**Create URL & Send GET Request**

In [57]:
radius = 500
limit = 2

def get_nearby_venues(df):
    venues_list = []
    
    for lat, lng, label in zip(df["Latitude"], df["Longitude"], df["PostalCode"]): 
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, 
            radius, limit)
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(
            label,
            lat,
            lng,
            venue["venue"]["name"],
            venue["venue"]["location"]["lat"],
            venue["venue"]["location"]["lng"],
            venue["venue"]["categories"][0]["name"]
        ) for venue in results])
    
    nearby_venues = pd.DataFrame([item for venue in venues_list for item in venue])
    nearby_venues.columns = ["PostalCode", "BoroughLatitude", "BoroughLongitude", 
                             "VenueName", "VenueLatitude", "VenueLongitude", "VenueCategory"]
    
    return nearby_venues

In [60]:
nearby_venues = get_nearby_venues(toronto_data)

**Examine Results**

In [70]:
print(nearby_venues.shape)
print("There are {} unique venue categories.\n".format(len(nearby_venues["VenueCategory"].unique())))
nearby_venues.head()

(76, 7)
There are 49 unique venue categories.



,PostalCode,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
3,M4K,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
4,M4L,43.668999,-79.315572,British Style Fish & Chips,43.668723,-79.317139,Fish & Chips Shop


In [69]:
nearby_venues.groupby("PostalCode").count()

There are 49 unique venue categories.



,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,,,,,,
M4E,2,2,2,2,2,2
M4K,2,2,2,2,2,2
M4L,2,2,2,2,2,2
M4M,2,2,2,2,2,2
M4N,2,2,2,2,2,2
M4P,2,2,2,2,2,2
M4R,2,2,2,2,2,2
M4S,2,2,2,2,2,2
M4T,2,2,2,2,2,2


#### Use One-Hot Encoding to Convert Categories into Numerical Values

In [101]:
nearby_venues_onehot = pd.get_dummies(nearby_venues["VenueCategory"])
nearby_venues_onehot["PostalCode"] = nearby_venues["PostalCode"]

# Re-order columns
cols = [nearby_venues_onehot.columns[-1]] + list(nearby_venues_onehot.columns.values[:-1])
nearby_venues_onehot = nearby_venues_onehot[cols]

nearby_venues_onehot.head()

,PostalCode,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Bar,Brewery,Bubble Tea Shop,Building,Burrito Place,...,Restaurant,Sandwich Place,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Theme Restaurant,Trail,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
nearby_venues_onehot.shape

(76, 50)

#### Group by PostalCode to get Frequency

In [106]:
nearby_venues_grouped = nearby_venues_onehot.groupby("PostalCode").mean().reset_index()
nearby_venues_grouped

,PostalCode,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Bar,Brewery,Bubble Tea Shop,Building,Burrito Place,...,Restaurant,Sandwich Place,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Theme Restaurant,Trail,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
5,M4P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M4R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
7,M4S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M4T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
9,M4V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0


#### Get Top 5 Most Common Categories for each Postal Code

In [128]:
def get_top_categories(idx, num=5):
    top = nearby_venues_grouped.iloc[idx, 1:].T.reset_index()
    top.columns = ["VenueCategory", "Frequency"]
    top["Frequency"] = top["Frequency"].astype(float)
    top = top.sort_values("Frequency", ascending=False).head(num)

In [129]:
for idx, row in nearby_venues_grouped.iterrows():
    get_top_categories(idx)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pand